In [1]:
from pprint import pprint

In [2]:
from hyperliquid.info import Info
from hyperliquid.utils import constants

info = Info(constants.TESTNET_API_URL, skip_ws=True)
user_state = info.user_state("0xcd5051944f780a621ee62e39e493c489668acf4d")
print(user_state)

{'assetPositions': [{'position': {'coin': 'SOL', 'cumFunding': {'allTime': '-179.642689', 'sinceChange': '-179.642689', 'sinceOpen': '-179.642689'}, 'entryPx': '22.2863', 'leverage': {'type': 'cross', 'value': 20}, 'liquidationPx': None, 'marginUsed': '430.818615', 'maxLeverage': 50, 'positionValue': '8616.37231', 'returnOnEquity': '13.45665675', 'szi': '126.49', 'unrealizedPnl': '5797.378223'}, 'type': 'oneWay'}, {'position': {'coin': 'APT', 'cumFunding': {'allTime': '-38.047495', 'sinceChange': '-38.047495', 'sinceOpen': '-38.047495'}, 'entryPx': '9.8095', 'leverage': {'type': 'cross', 'value': 20}, 'liquidationPx': None, 'marginUsed': '23.815556', 'maxLeverage': 50, 'positionValue': '476.311131', 'returnOnEquity': '-2.27938092', 'szi': '54.09', 'unrealizedPnl': '-54.284724'}, 'type': 'oneWay'}, {'position': {'coin': 'ATOM', 'cumFunding': {'allTime': '-1747.034874', 'sinceChange': '0.460828', 'sinceOpen': '-1747.034874'}, 'entryPx': '10.9725', 'leverage': {'type': 'cross', 'value': 2

In [3]:
import json

import eth_account
import utils
from eth_account.signers.local import LocalAccount

from hyperliquid.exchange import Exchange
from hyperliquid.info import Info
from hyperliquid.utils import constants


def main():
    config = utils.get_config()
    account: LocalAccount = eth_account.Account.from_key(config["secret_key"])
    print("Running with account address:", account.address)
    info = Info(constants.TESTNET_API_URL, skip_ws=True)

    # Get the user state and print out position information
    user_state = info.user_state(account.address)
    positions = []
    for position in user_state["assetPositions"]:
        if float(position["position"]["szi"]) != 0:
            positions.append(position["position"])
    if len(positions) > 0:
        print("positions:")
        for position in positions:
            print(json.dumps(position, indent=2))
    else:
        print("no open positions")

    # Place an order that should rest by setting the price very low
    exchange = Exchange(account, constants.TESTNET_API_URL)
    order_result = exchange.order("ETH", True, 0.2, 1100, {"limit": {"tif": "Gtc"}})
    print(order_result)

    # Query the order status by oid
    if order_result["status"] == "ok":
        status = order_result["response"]["data"]["statuses"][0]
        if "resting" in status:
            order_status = info.query_order_by_oid(account.address, status["resting"]["oid"])
            print("Order status by oid:", order_status)

    # Cancel the order
    if order_result["status"] == "ok":
        status = order_result["response"]["data"]["statuses"][0]
        if "resting" in status:
            cancel_result = exchange.cancel("ETH", status["resting"]["oid"])
            print(cancel_result)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'utils'

# Aevo

In [ ]:
'https://api-testnet.aevo.xyz'

In [10]:
import dotenv
import os

# Load the .env file
dotenv.load_dotenv()

# Get the AEVO_TESTNET_SIGNING_KEY from the environment variables
# signing_key = os.getenv("AEVO_TESTNET_SIGNING_KEY")

# Print the signing key
# print(signing_key)


In [12]:
# signing_key

In [ ]:
from random import randint
from eip712_structs import EIP712Struct, Address, Uint, Boolean, make_domain
from web3 import Web3
from eth_account import Account

# Generate Salt
salt = randint(0, 10**6)

# Limit price and amount is in 6 decimal places
decimals = 10**6
limit_price = int(100 * decimals)  # Limit price of $100
amount = int(2 * decimals)  # Size of 2 contracts

# Current timestamp in UNIX seconds
timestamp = 1690434000 

# instrument_id from /options-chain
instrument = 1

class Order(EIP712Struct):
    maker = Address()
    isBuy = Boolean()
    limitPrice = Uint(256)
    amount = Uint(256)
    salt = Uint(256)
    instrument = Uint(256)
    timestamp = Uint(256)

order_struct = Order(maker="0xAf553Fa01c1d7c1677E35d8BB1b81982B1711002", # The wallet's main address
                    isBuy=True, # True if buy, False if sell
                    limitPrice=limit_price,
                    amount=amount,
                    salt=salt,
                    instrument=instrument,
                    timestamp=timestamp)

# Get bytes
# domain = make_domain(name="Aevo Mainnet", version="1", chainId=1)
# Testnet Domain
domain = make_domain(name='Aevo Testnet', version='1', chainId=11155111)
signable_bytes = Web3.keccak(order_struct.signable_bytes(domain=domain))

# Sign with key
key = signing_key
signature = Account._sign_hash(signable_bytes, key).signature.hex()

In [ ]:
signature

'0x4706131697eaae8b17411d6557f3dff2361e6291fb05509da2fd44076d1990382c668c35544f112f5dde40959bb9817c08dcfc6517f7ef772397e6474796d7071b'

In [ ]:
from random import randint
from eip712_structs import EIP712Struct, Address, Uint, Boolean, make_domain
from web3 import Web3
from eth_account import Account

# Generate Salt
salt = randint(0, 10**6)

# Limit price and amount is in 6 decimal places
decimals = 10**6
limit_price = int(100 * decimals)  # Limit price of $100
amount = int(2 * decimals)  # Size of 2 contracts

# instrument_id from /options-chain
instrument = 1

class Order(EIP712Struct):
    maker = Address()
    isBuy = Boolean()
    limitPrice = Uint(256)
    amount = Uint(256)
    salt = Uint(256)
    instrument = Uint(256)

order_struct = Order(maker="0xAf553Fa01c1d7c1677E35d8BB1b81982B1711002", # The wallet's main address
                    isBuy=True, # True if buy, False if sell
                    limitPrice=limit_price,
                    amount=amount,
                    salt=salt,
                    instrument=instrument)

# Get bytes
# domain = make_domain(name="Aevo Mainnet", version="1", chainId=1)
# Testnet Domain
domain = make_domain(name='Aevo Testnet', version='1', chainId=11155111)
signable_bytes = Web3.keccak(order_struct.signable_bytes(domain=domain))

# Sign with key
key = signing_key
signature = Account._sign_hash(signable_bytes, key).signature.hex()

In [4]:
signature

NameError: name 'signature' is not defined

In [5]:
# Code Example

import os
import time
import json
import hmac
import hashlib
import requests

API_KEY = os.environ["AEVO_TESTNET_API"]
API_SECRET = os.environ["AEVO_TESTNET_SECRET"]
# print(API_KEY)
# print(API_SECRET)

def get_headers(path, method, body):
    timestamp = str(time.time_ns())

    # If the body is empty, it would look like:
    # concat = 'API_KEY,1673425955575713842,GET,/account,'
    concat = f"{API_KEY},{timestamp},{method.upper()},{path},{body}".encode("utf-8")
    signature = hmac.new(API_SECRET.encode("utf-8"), concat, hashlib.sha256).hexdigest()

    headers = {
        "AEVO-TIMESTAMP": timestamp,
        "AEVO-SIGNATURE": signature,
        "AEVO-KEY": API_KEY,
    }

    return headers


# GET method - you pass in an empty string as the body
headers = get_headers("/account", "GET", "")
req = requests.get("https://api-testnet.aevo.xyz/account", headers=headers)
print(req.json())

# POST method - you have to pass in the JSON string used in the request body
body = json.dumps({"name": "My API key"})
print(body)
headers = get_headers("/api-key", "POST", body)
req = requests.post("https://api-testnet.aevo.xyz/api-keys",
                    headers=headers,
                    data=body)
print(req.json())

KeyError: 'AEVO_TESTNET_API'

In [13]:
API_KEY = os.environ["AEVO_TESTNET_API"]
API_SECRET = os.environ["AEVO_TESTNET_SECRET"]
SIGNING_KEY = os.environ["AEVO_TESTNET_SIGNING_KEY"]

In [14]:
import requests

url = "https://api-testnet.aevo.xyz/markets"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

[{'amount_step': '100',
  'index_price': '0.012129',
  'instrument_id': '12623',
  'instrument_name': '1000BONK-PERP',
  'instrument_type': 'PERPETUAL',
  'is_active': True,
  'mark_price': '0.012129',
  'max_leverage': '5',
  'min_order_value': '10',
  'price_step': '0.000001',
  'quote_asset': 'USDC',
  'underlying_asset': '1000BONK'},
 {'amount_step': '0.001',
  'index_price': '41368.747014',
  'instrument_id': '3657',
  'instrument_name': 'BTC-PERP',
  'instrument_type': 'PERPETUAL',
  'is_active': True,
  'mark_price': '41371.030008',
  'max_leverage': '20',
  'min_order_value': '0.01',
  'price_step': '0.1',
  'quote_asset': 'USDC',
  'underlying_asset': 'BTC'},
 {'amount_step': '1',
  'index_price': '0.639914',
  'instrument_id': '6391',
  'instrument_name': 'CRV-PERP',
  'instrument_type': 'PERPETUAL',
  'is_active': True,
  'mark_price': '0.6403',
  'max_leverage': '20',
  'min_order_value': '0.001',
  'price_step': '0.001',
  'quote_asset': 'USDC',
  'underlying_asset': 'CRV'

In [16]:
import requests

url = "https://api-testnet.aevo.xyz/orders"

payload = {
    "instrument": 4561,
    "maker": "0xAf553Fa01c1d7c1677E35d8BB1b81982B1711002",
    "is_buy": True,
    "amount": "10",
    "limit_price": 252,
    "salt": "12345678",
    "signature": SIGNING_KEY,
    "timestamp": "1680249600",
    "post_only": False,
    "reduce_only": False,
    "time_in_force": "GTC",
    "mmp": False,
    "stop": "STOP_LOSS",
    "trigger": 200,
    "close_position": False
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "AEVO-KEY": API_KEY,
    "AEVO-SECRET": API_SECRET
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"error":"ORDER_BAD_SIGNING_KEY"}
